# Carnivore

## Description

This notebook compares the world's most popular meat production (beef, chicken, and swine/pig) to the world's population over time (from the year 1961 to the year 2021). The goal is to determine whether food production is keeping up with global population growth.

## Meat Production

The world's meat production data was gathered from the [Food and Agriculture Organization of the United Nations](https://www.fao.org/faostat/en/#data/QCL). The following image displays the constraints used to derive the CSV of beef, chicken, and swine data:

![FAO Data Constraints](FAO_data_constraints.png "FAO Data Constraints")

## Population

The world's population data was gathered from the [United Nations' Data Portal of the Population Division](https://population.un.org/dataportal/data/indicators/49/locations/900/start/1961/end/2021/table/pivotbylocation). The following image displays the constraints used to derive the CSV of total human population:

![Population Data Constraints](population_data_constraints.png "Population Data Constraints")

hello
